In [ ]:
!pip install spectral
import spectral.io.envi as envi
from spectral import *

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import os
base_path = os.path.abspath('')

In [ ]:
img2 = envi.open(base_path + '/Data/F15_20160101_20161231.v4.stable_lights.avg_vis.hdr', base_path + '/Data/F15_20160101_20161231.v4.stable_lights.avg_vis')
img_old = img2.load()
img_old = img_old.reshape(16801, 43201)
np.around(img_old, 0, out=img_old)
np.max(img_old)

In [ ]:
import pandas as pd

# ['model_05','model_06','model_07','model_08', 'model_09', 'model_10', 'model_11', 'model_12', 'model_13', 'model_14', 'model_15', 'model_16', 
#                                                             'model_17','model_18', 'model_19', 'model_20', 'model_21', 'model_22', 'model_23']
# these are all possible models (in my case) 

models_to_estimate = ['model_18', 'model_19', 'model_20', 'model_21', 'model_22', 'model_23']

df_coords =  pd.read_pickle(base_path + '/Data/spots_of_interest.pkl')
indexies = ['MAE_all map', 'MSE_all map','L_infinity_norm_all map']
for i in df_coords.columns:
  indexies.append('MAE_{}'.format(i))
  indexies.append('MSE_{}'.format(i))
  indexies.append('R2_{}'.format(i))
  indexies.append('L_infinity_norm_{}'.format(i))
df_accuracy = pd.DataFrame(np.nan, index=indexies, columns=models_to_estimate)
df_accuracy

In [ ]:
df_coords

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, max_error


for model in models_to_estimate:
  model_number = int(model[-2:])
  prediction = np.load(base_path + '/Predictions/pred{}.npz'.format(model_number))['arr_0']
  df_accuracy.iloc[0].loc[model], df_accuracy.iloc[1].loc[model] = mean_absolute_error(img_old, prediction), mean_squared_error(img_old, prediction)
  df_accuracy.iloc[2].loc[model] = max_error(img_old.reshape(-1), prediction.reshape(-1))
  print(model_number - int(models_to_estimate[0][-2:]) + 1, '/', len(models_to_estimate), 'started')
  for j in range(1,6):
    y = df_coords.iloc[0,j-1]
    x = df_coords.iloc[1,j-1]
    df_accuracy.iloc[j*4-1].loc[model], df_accuracy.iloc[j*4].loc[model], df_accuracy.iloc[j*4+1].loc[model] = mean_absolute_error(img_old[y-210:y, x-300:x], prediction[y-210:y, x-300:x]), mean_squared_error(img_old[y-210:y, x-300:x], prediction[y-210:y, x-300:x]), r2_score(img_old[y-210:y, x-300:x], prediction[y-210:y, x-300:x])
    df_accuracy.iloc[j*4+2].loc[model] = max_error(img_old[y-210:y, x-300:x].reshape(-1), prediction[y-210:y, x-300:x].reshape(-1))

df_accuracy

In [ ]:
prediction = np.load(base_path + '/Predictions/pred{}.npz'.format(22))['arr_0']

In [ ]:
from skimage.io import imsave
from skimage.io import call_plugin, use_plugin, find_available_plugins
use_plugin('imageio')

In [ ]:
y = df_coords.iloc[0,3]
x = df_coords.iloc[1,3]
imshow(img_old[y-210:y, x-300:x])
imshow(prediction[y-210:y, x-300:x])

In [ ]:
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")
for model in models_to_estimate:
  model_number = int(model[-2:])
  prediction = np.load(base_path + '/Predictions/pred{}.npz'.format(model_number))['arr_0']
  for j in range(1, 6):
    y = df_coords.iloc[0,j-1]
    x = df_coords.iloc[1,j-1]
    spot_name = df_coords.columns[j-1]
    imsave(base_path + '/Predictions/preds_for_spots/{}_{}.jpg'.format(spot_name, model_number), prediction[y-210:y, x-300:x], quality=75)

    orig = img_old[y-210:y, x-300:x].reshape(-1, 1)
    pred = prediction[y-210:y, x-300:x].reshape(-1, 1)
    data = pd.DataFrame(data=np.concatenate((orig, pred), axis=1), columns=['Original', 'Predicted'])
    data = data.groupby(by=['Original', 'Predicted'])['Predicted'].count().reset_index(name='Count')
    data['Sum'] = data.groupby('Original')['Count'].transform('sum')
    data['Fraction'] = data.Count / data.Sum

    x_bins = np.linspace(0,63,64)
    y_bins = np.linspace(0,63,64)
    f, ax = plt.subplots(figsize=(8, 8))
    sns.despine(f)
    cmp = sns.cubehelix_palette(rot=0.8, start=0, as_cmap=True)
    sns_plot = sns.scatterplot(x='Original', y='Predicted', data=data, hue='Fraction',hue_norm=(0,1), size='Count', x_bins=x_bins, y_bins=y_bins, ax=ax, palette=cmp)
    figure = sns_plot.get_figure()
    figure.savefig(base_path + '/Predictions/preds_for_spots/{}_{}_plot.jpg'.format(spot_name, model_number))

In [ ]:
np.max(prediction[y-110:y-60, x-175:x-125])

In [ ]:
# for j in range(1, 6):
#     y = df_coords.iloc[0,j-1]
#     x = df_coords.iloc[1,j-1]
#     spot_name = df_coords.columns[j-1]
#     #imshow(img_old[y-210:y, x-300:x])
#     #imshow(prediction[y-210:y, x-300:x])
#     imsave('/content/drive/My Drive/Diploma/Predictions/preds_for_spots/{}_{}.jpg'.format(spot_name, 'ground_truth'), img_old[y-210:y, x-300:x], quality=75)

In [ ]:
y = df_coords.iloc[0,2]
x = df_coords.iloc[1,2]
from sys import getsizeof
orig = img_old[y-210:y, x-300:x].reshape(-1, 1)
pred = prediction[y-210:y, x-300:x].reshape(-1, 1)
print(getsizeof(orig) / (1024 * 1024))
# orig = orig[orig != 0]
# pred = pred[pred != 0]
# print(getsizeof(orig) / (1024 * 1024))

# img_old = None
# prediction = None
data = pd.DataFrame(data=np.concatenate((orig, pred), axis=1), columns=['Original', 'Predicted'])
#data = data.loc[(data.Original != 0) & (data.Predicted != 0)]
data = data.groupby(by=['Original', 'Predicted'])['Predicted'].count().reset_index(name='Count')
data['Sum'] = data.groupby('Original')['Count'].transform('sum')
data['Fraction'] = data.Count / data.Sum
data.head()

In [ ]:
x_bins = np.linspace(0,63,64)
y_bins = np.linspace(0,63,64)
f, ax = plt.subplots(figsize=(8, 8))
sns.despine(f)
cmp = sns.cubehelix_palette(rot=0.8, start=0, as_cmap=True)
sns_plot = sns.scatterplot(x='Original', y='Predicted', data=data, hue='Fraction',hue_norm=(0,1), x_bins=x_bins, y_bins=y_bins, ax=ax, palette=cmp, marker='s')
# figure = sns_plot.get_figure()
# figure.savefig('output.png')
# sns.regplot(x='Original', y='Predicted', data=data, ax=ax, marker='s')

In [ ]:
import seaborn as sns; sns.set(color_codes=True)
tips = sns.load_dataset("tips") sns.lmplot(x="total_bill", y="tip", data=tips)